In [40]:
import geopandas as gpd
import pandas as pd

## Open GIS Data

In [41]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Age data

In [42]:
acs_marriage = pd.read_csv(
    "data/ACSST5Y2023.S1201-2025-05-02T023159.csv", thousands=","
)

In [43]:
acs_marriage = acs_marriage.T.reset_index()
acs_marriage.columns = acs_marriage.iloc[0]
acs_marriage = acs_marriage.iloc[1:]
state_col = acs_marriage.columns[0]
marriage_col = acs_marriage.columns[-3]
acs_marriage = acs_marriage[[state_col, marriage_col]]
acs_marriage = acs_marriage.rename(columns={state_col: "STATE", marriage_col: "RATIO"})

In [44]:
acs_marriage = acs_marriage[acs_marriage["RATIO"] != "(X)"]
acs_marriage["NAME"] = acs_marriage["STATE"].str.split("!!").str[0]
acs_marriage["RATIO"] = acs_marriage["RATIO"].astype(float)

## Combine Data

In [45]:
gender_gdf = states_df.merge(acs_marriage, on="NAME", how="left")

In [47]:
gender_gdf = gender_gdf[["geometry", "NAME", "RATIO"]].dropna()

In [48]:
gender_gdf = gender_gdf.to_crs(9311)

In [49]:
gender_gdf.to_file("data/gender.gpkg")